In [1]:
import os

train_filepath = 'train'
test_filepath = 'test'

train_dir = os.listdir(train_filepath)
test_dir = os.listdir(test_filepath)


In [2]:
t_hotel_dir = os.listdir(os.path.join(train_filepath,'hotel'))
t_travel_dir = os.listdir(os.path.join(train_filepath,'travel'))


In [3]:
t_hotel = []
t_travel = []
for file in t_hotel_dir:
    with open(os.path.join(train_filepath,'hotel',file),'r',encoding='utf-8') as f:
        lines = f.readlines()
        tmp_str = ''
        for line in lines:
            if line.find('www') ==-1 and line.find('com') ==-1 and line.find('保存时间') and line.find('电') and line.find('传') ==-1 :
                tmp_str += line.strip()
        t_hotel.append(tmp_str)
for file in t_travel_dir:
    with open(os.path.join(train_filepath,'travel',file),'r',encoding='utf-8') as f:
        lines = f.readlines()
        tmp_str = ''
        for line in lines:
            if line.find('www') ==-1 and line.find('com') ==-1:
                tmp_str += line.strip()
        t_travel.append(tmp_str)


In [4]:
import jieba

stopwords = {}.fromkeys(['，','。','!',':',"-","、","：","；"," ",'这','我','非常'])

corpus_h = []
for hotel in t_hotel:
    seg_list = jieba.cut(hotel,cut_all=False)      
    final = ''
    for seg in seg_list:
        if seg not in stopwords:  
            final +=seg
            
    seg_list = jieba.cut(final,cut_all=False)
    output = ' '.join(list(seg_list))
    corpus_h.append(output)

corpus_t = []
for travel in t_travel:
    seg_list = jieba.cut(travel,cut_all=False)       
    final = ''
    for seg in seg_list:
        if seg not in stopwords:  
            final +=seg
            
    seg_list = jieba.cut(final,cut_all=False)
    output = ' '.join(list(seg_list))
    corpus_t.append(output)


Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/sj/y6l66yt16v1drk7qcyx6sd3c0000gn/T/jieba.cache
Loading model cost 0.795 seconds.
Prefix dict has been built succesfully.


In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer() 
corpus = corpus_h + corpus_t
X = vectorizer.fit_transform(corpus) 

word = vectorizer.get_feature_names()  

X = X.toarray()

['00',
 '0000000',
 '01460112040',
 '02871219867115858addzhulinsidujiangyantel0287121986postalcode611830',
 '029',
 '03606006020',
 '04',
 '05',
 '051',
 '05125206221',
 '05147315888',
 '05147325288',
 '05147342611',
 '05147344467',
 '05147344888',
 '05147888333',
 '05592510001',
 '05592511067',
 '05592512585',
 '05592512985',
 '05592513561',
 '05592526999',
 '05592527678',
 '057471168881247124952',
 '05747121688',
 '05747250088emailnbdjd',
 '05747293175',
 '05747757888',
 '06',
 '067000',
 '067002',
 '07',
 '0732',
 '07966552257',
 '0800',
 '0837',
 '0872',
 '08722670406',
 '088851209296',
 '08885124309',
 '08885129951',
 '08885175382',
 '08885187160',
 '08885187619',
 '08986532222608986532222513976380766',
 '0898882780800898882780805050',
 '0899',
 '0996',
 '0k',
 '0km',
 '10',
 '100',
 '1000',
 '10000',
 '100020',
 '1000558558',
 '100150',
 '100180',
 '1007',
 '101',
 '1010',
 '10120',
 '1015',
 '1015750750',
 '1018',
 '102',
 '1020488488',
 '1021420420',
 '1021440440',
 '1022',
 '1

In [8]:
import numpy as np
label = [1 for i in range(308)] + [0 for i in range(308)]
New_X = np.column_stack((X,label))
np.random.shuffle(New_X)

In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

x_train = New_X[:500][:,:-1]
x_train_l = New_X[:500][:,-1]
x_test = New_X[500:][:,:-1]
x_test_l = New_X[500:][:,-1]



clf = MultinomialNB().fit(x_train,x_train_l)
pre = clf.predict(x_test)
print("预测结果:",pre)
print("真实结果:",x_test_l)

print(np.sum(x_test_l==pre)/len(x_test))

预测结果: [0 0 1 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 1 0 1 1 1 0 0 1 1 0 1 1 1
 1 1 0 0 1 1 0 1 1 1 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0 1 0
 0 1 1 0 0 0 1 1 0 0 1 0 0 1 0 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 0 1 0 1 0 0
 0 0 1 1 0]
真实结果: [0 0 1 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 1 0 1 1 1 0 0 1 1 0 1 1 1
 1 1 0 0 1 1 0 1 1 1 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0 1 0
 0 1 1 0 0 0 1 1 1 0 1 0 0 1 0 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1 0 1 0 0
 0 0 1 1 0]
0.9827586206896551


In [10]:
final_test_data = []

for file in test_dir:
    with open(os.path.join(test_filepath,file),'r',encoding='utf-8') as f:
        lines = f.readlines()
        tmp_str = ''
        for line in lines:
            if line.find('www') ==-1 and line.find('com') ==-1 and line.find('保存时间') and line.find('电') and line.find('传') ==-1 :
                tmp_str += line.strip()
        final_test_data.append(tmp_str)
print(test_dir)
corpus_final = []
for test in final_test_data:
    seg_list = jieba.cut(test,cut_all=False)       
    final = ''
    for seg in seg_list:
        if seg not in stopwords:  
            final +=seg
            
    seg_list = jieba.cut(final,cut_all=False)
    output = ' '.join(list(seg_list))
    corpus_final.append(output)

Y = vectorizer.transform(corpus_final)
Y = Y.toarray()
pre = clf.predict(Y)
print(pre)

['guangzhou10_seg_pos.txt', '三亚市春节宾馆房价不乱涨价违者将受到严处_seg_pos.txt', '住宿-宾馆名录_seg_pos.txt', 'qufu03_seg_pos.txt', 'nj12_seg_pos.txt', 'nj7_seg_pos.txt', 'dali09_seg_pos.txt', 'bj6_seg_pos.txt', 'xm7_seg_pos.txt', 'dujiangyan09_seg_pos.txt', 'haerbin13_seg_pos.txt', 'bj15_seg_pos.txt', 'zhuhai15_seg_pos.txt', 'xm12_seg_pos.txt', 'bj4_seg_pos.txt', 'wuyishan12_seg_pos.txt', 'xian03_seg_pos.txt', 'bj1_seg_pos.txt', 'nj9_seg_pos.txt', 'zhuhai06_seg_pos.txt', 'kuerle01_seg_pos.txt', 'xm3_seg_pos.txt']
[0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 1 0 0 0 1 1 1]


In [21]:
#输出类别个数
print('hotel:%d,travel:%d'%(np.sum(pre),len(test_dir)-np.sum(pre)))

#输出类别
print('hotel:')
for i in range(len(pre)):
    if pre[i] == 1:
        print(test_dir[i])
print('travel:')
for i in range(len(pre)):
    if pre[i] == 0:
        print(test_dir[i])
    

hotel:11,travel:11
hotel:
三亚市春节宾馆房价不乱涨价违者将受到严处_seg_pos.txt
住宿-宾馆名录_seg_pos.txt
nj7_seg_pos.txt
dali09_seg_pos.txt
bj6_seg_pos.txt
xm7_seg_pos.txt
dujiangyan09_seg_pos.txt
wuyishan12_seg_pos.txt
zhuhai06_seg_pos.txt
kuerle01_seg_pos.txt
xm3_seg_pos.txt
travel:
guangzhou10_seg_pos.txt
qufu03_seg_pos.txt
nj12_seg_pos.txt
haerbin13_seg_pos.txt
bj15_seg_pos.txt
zhuhai15_seg_pos.txt
xm12_seg_pos.txt
bj4_seg_pos.txt
xian03_seg_pos.txt
bj1_seg_pos.txt
nj9_seg_pos.txt
